# Etapa 1: o espectro das trilhas de áudio

In [1]:
from configuration import *
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
import pickle
import time

#%matplotlib inline

In [2]:
#df = pd.read_csv(DETECTIONS, delimiter="|", nrows=n_records)
df = pd.read_csv('tests/2-tracks_6-features.csv')

if not n_records:
    n_records = len(df)

print("Usaremos {} registros.".format(n_records))

Usaremos 5000 registros.


O espectro de cada trilha de áudio pode ser obtido criando uma tabela pivô, agrupando os dados nas linhas por <code>track_id</code>, as colunas por <code>audio_source_id</code> e contabilizando as ocorrências (coluna <code>created_at</code>) como função de agregação. Como resultado, a tabela pivô apresentará as <code>track_id</code> nas linhas (sem repetição) e a quantidade de vezes em que ela foi tocada em cada rádio, nas colunas.

In [3]:
start = time.time()

pt = pd.pivot_table(df,
                    values='created_at',
                    index='track_id',
                    columns=['audio_source_id'],
                    aggfunc='count',
                    fill_value=0)

O código abaixo interpreta a tabela pivô como uma matriz e transforma-a para frequências relativas:

In [4]:
b = pt.as_matrix()
tracks = np.divide(b, np.sum(b, axis=1, dtype=np.float64)[:, np.newaxis])

In [6]:
b

array([[3, 2, 4, 0, 3, 0],
       [0, 0, 0, 8, 0, 4]])

In [25]:
end = time.time()
print("Levou {} s para calcular todos os espectros.".format(end-start))

Levou 9.49830007553 s para calcular todos os espectros.


In [15]:
n_samples, n_features = np.shape(tracks)
print("{} amostras x {} dimensões (features).".format(n_samples, n_features))

2 amostras x 6 dimensões (features).


###  Persistência dos espectros

Persiste os espectros num formato compactado e apropriado para o algoritmo de classificação:

In [16]:
assert not np.any(np.isnan(tracks))
assert np.max(tracks) <= 1.0
assert np.min(tracks) >= 0.0

In [17]:
sparse_matrix = csr_matrix(tracks)
np.savez(TRACKS,
         data = sparse_matrix.data,
         indices = sparse_matrix.indices,
         indptr = sparse_matrix.indptr,
         shape = sparse_matrix.shape)

### Persistência dos track_ids

A lista de track_ids ordenada será necessária no próximo estágio para associar cada track_id com um ID de gênero musical. Para não ter de criar novamente essa lista, salvamo-la em disco:

In [18]:
pickle.dump(pt.index, file(TRACK_IDS, 'wb'), pickle.HIGHEST_PROTOCOL)